# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [1]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [2]:
!cat talks.tsv


"Stabilisation of exact coherent structures using time-delayed feedback in 2D turbulence 
"	Talk	Talk21	"Numerical Analysis Seminar, North Carolina State University 

"Stabilisation of exact coherent structures using time-delayed feedback in 2D turbulence 
"	Talk	Talk20	"Shear flow transition and turbulence seminar

"Stabilisation of exact coherent structures using time-delayed feedback in 2D turbulence 

"Stabilisation of exact coherent structures using time-delayed feedback in 2D turbulence 

"Layers, localisation, and relaminarisation in horizontally shearing stratified turbulence 

"Layers, localisation, and relaminarisation in horizontally shearing stratified turbulence 

"Layers, localisation, and relaminarisation in horizontally shearing stratified turbulence 

"Layers, localisation, and relaminarisation in horizontally shearing stratified turbulence 

"Layers, localisation, and relaminarisation in horizontally shearing stratified turbulence 











"Recurrent flows embedde

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description,Unnamed: 8
0,Stabilisation of exact coherent structures usi...,Talk,Talk21,"Numerical Analysis Seminar, North Carolina Sta...",2021-10-19,Online,NaN,NaN,NaN
1,Stabilisation of exact coherent structures usi...,Talk,Talk20,Shear flow transition and turbulence seminar\n,2021-07-23,Online,NaN,NaN,NaN
2,Stabilisation of exact coherent structures usi...,Talk,Talk19,Australasian fluid mechanics seminar series,2020-11-25,Online,https://www.youtube.com/watch?v=CeQWeZ7vNzU,NaN,NaN
3,Stabilisation of exact coherent structures usi...,Talk,Talk18,"Physical applied mathematics seminar, Universi...",2020-11-11,Online,NaN,NaN,NaN
4,"Layers, localisation, and relaminarisation in ...",Conference talk,Talk17,"17th European Turbulence Conference, Turin",2019-07-03,"Turin, Italy",http://www.etc17.it/,NaN,NaN
5,"Layers, localisation, and relaminarisation in ...",Talk,Talk16,"Fluids & Materials Seminar, University of Bristol",2019-05-02,"Bristol, UK",NaN,NaN,NaN
6,"Layers, localisation, and relaminarisation in ...",Talk,Talk15,"Aerodynamics & Flight Mechanics Group Seminar,...",2019-05-08,"Southampton, UK",NaN,NaN,NaN
7,"Layers, localisation, and relaminarisation in ...",Talk,Talk14,"Applied Mathematics Seminar, Coventry University",2018-11-07,"Coventry, UK",NaN,NaN,NaN
8,"Layers, localisation, and relaminarisation in ...",Talk,Talk13,Woods Hole Geophysical Fluid Dynamics Summer P...,2018-07-03,"Woods Hole, MA, USA",https://www.whoi.edu/gfd/,NaN,NaN
9,Layers & mixing in stratified turbulence; a dy...,Talk,Talk12,"Fluids & MHD Seminar, University of Leeds",2018-05-24,"Leeds, UK",NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
loc_dict = {}

for row, item in talks.iterrows():
    
    md_filename = str(item.date) + "-" + item.url_slug + ".md"
    html_filename = str(item.date) + "-" + item.url_slug 
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [6]:
!ls ../_talks

2012-03-01-talk-1.md     2017-08-21-Talk9.md      2019-07-03-Talk17.md
2013-03-01-tutorial-1.md 2018-02-09-Talk11.md     2019-07-03-Talk5.md
2014-02-01-talk-2.md     2018-05-24-Talk12.md     2020-11-11-Talk18.md
2014-03-01-talk-3.md     2018-07-02-Talk10.md     2020-11-11-Talk6.md
2014-03-25-Talk1.md      2018-07-03-Talk1.md      2020-11-25-Talk19.md
2014-11-26-Talk3.md      2018-07-03-Talk13.md     2020-11-25-Talk7.md
2014-12-09-Talk4.md      2018-11-07-Talk14.md     2021-07-23-Talk20.md
2015-05-21-Talk2.md      2018-11-07-Talk2.md      2021-07-23-Talk8.md
2015-08-26-Talk5.md      2019-05-02-Talk16.md     2021-10-19-Talk21.md
2016-03-02-Talk6.md      2019-05-02-Talk4.md      2021-10-19-Talk9.md
2016-10-12-Talk7.md      2019-05-08-Talk15.md     talks.html
2016-11-20-Talk8.md      2019-05-08-Talk3.md      talks.html~


In [7]:
!cat ../_talks/2020-11-11-Talk6.md

---
title: "Stabilisation of exact coherent structures using time-delayed feedback in 2D turbulence 
"
collection: talks
type: "Talk"
permalink: /talks/2020-11-11-Talk6
venue: "Physical applied mathematics seminar, University of Manchester"
date: 2020-11-11
location: "Online"
---
